In [18]:
import pandas as pd
from coordenadas import obter_coordenadas, uf
import sqlite3
from sqlalchemy import create_engine


df_municipios = pd.read_csv('parahybaCities.csv', engine = 'python')

df_municipios = df_municipios.drop(columns=['Tecnologia', 'Tipo de compromisso', 'Compromisso', 'Origem compromisso', 'Operadora com compromisso', 'Status compromisso',
                               'Prazo compromisso', 'Operadora'], axis= 1)

df_municipios = df_municipios.drop_duplicates()

df_municipios['Latitude'] = ''
df_municipios['Longitude'] = ''


for i, municipio in enumerate (df_municipios['Município']):
    lat, lon = obter_coordenadas(municipio, uf)
    df_municipios.at[i, 'Latitude'] = lat
    df_municipios.at[i, 'Longitude'] = lon


df_municipios['% moradores cobertos'] = df_municipios['% moradores cobertos'].str.replace('%', '').str.replace(',', '.').str.strip()
df_municipios['% moradores cobertos'] = pd.to_numeric(df_municipios['% moradores cobertos'], errors='coerce')
df_municipios['% moradores cobertos'] = df_municipios['% moradores cobertos'].fillna(0)

df_municipios['Longitude'] = pd.to_numeric(df_municipios['Longitude'], errors='coerce')
df_municipios['Latitude'] = pd.to_numeric(df_municipios['Latitude'], errors='coerce')


engine = create_engine('sqlite:///municipios_PB.db')
with engine.connect() as conn:
    df_municipios.to_sql('municipios', conn, if_exists='replace', index=False)


df_municipios = df_municipios.dropna(how='any')

display(df_municipios)

,Código IBGE,Município,UF,% área coberta,% moradores cobertos,% domicílios cobertos,Área km2,Moradores,Domicílios,Latitude,Longitude
0,2500106.0,Água Branca,PB,"43,22",63.36,"65,47","236,609529",9446.0,2559.0,-7.515105,-37.639403
2,2500205.0,Aguiar,PB,"39,33",64.71,"66,78","344,7094987",5528.0,1659.0,-7.041521,-35.628532
5,2500304.0,Alagoa Grande,PB,"67,75",93.55,"94,05","320,5623905",28460.0,8012.0,-7.741342,-36.065357
7,2500403.0,Alagoa Nova,PB,"97,82",99.23,"99,29","122,2553871",19678.0,5434.0,-7.430425,-34.910610
10,2500502.0,Alagoinha,PB,"98,21",99.57,"99,54","96,97979893",13533.0,3686.0,-6.785960,-38.083135
...,...,...,...,...,...,...,...,...,...,...,...
209,2504306.0,Catolé do Rocha,PB,"50,74",86.14,"86,46","552,1079469",28648.0,8227.0,-6.767694,-36.245508
213,2504355.0,Caturité,PB,"84,18",94.15,"94,38","118,0824652",4543.0,1329.0,-7.206294,-36.824499
217,2504405.0,Conceição,PB,"36,76",73.66,"75,68","579,4368234",18256.0,5112.0,-6.579042,-38.596986
219,2504504.0,Condado,PB,"66,95",89.74,"90,56","280,9152421",6524.0,1891.0,-7.696494,-35.666623


In [27]:
import plotly.express as px


grafico = px.density_mapbox(df_municipios, lon="Longitude", lat="Latitude", z="% moradores cobertos", mapbox_style="open-street-map", zoom = 7, radius=30, height=600)
grafico.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
grafico.show()